### Columns to drop
- building_metadata - drop floor count, year built
- weather_train - precipitation depth 1_hr, cloud coverage

### Sites to Drop
- see files for electricity, steam, gas

### SQL Query

In [20]:
import psycopg2
from psycopg2 import Error

import pandas as pd
import numpy as np

import os
path = os.path.join(os.path.expanduser('~'), 'Documents', 'github.passwords', 'energy.predictor.2020.config')
print (path)

myfile = open(path,"rt")
config= myfile.read()

/home/douglas/Documents/github.passwords/energy.predictor.2020.config


In [21]:
connection = psycopg2.connect(config)

cursor = connection.cursor()

In [22]:
# Read PostgreSQL into a dataframe
sql =''' SELECT 
            bm.site_id,
            bm.building_id,
            bm.primary_use,
            bm.square_feet,
            t.meter,
            t.meter_reading,
            w.timestamp,
            w.air_temperature,
            w.dew_temperature,
            w.sea_level_pressure,
            w.wind_direction,
            w.wind_speed
         FROM 
             weather_train AS w
         INNER JOIN building_metadata AS bm
             ON (bm.site_id = w.site_id)
         INNER JOIN train AS t
             ON (t.timestamp = w.timestamp) AND (t.building_id = bm.building_id)
         WHERE 
             w.timestamp IS NOT NULL 
             AND w.air_temperature IS NOT NULL 
             AND w.dew_temperature IS NOT NULL
             AND w.sea_level_pressure IS NOT NULL 
             AND w.wind_direction IS NOT NULL 
             AND w.wind_speed IS NOT NULL
            ;'''
df = pd.read_sql_query(sql,connection)

### Drop any observation that has a null

In [23]:
if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

PostgreSQL connection is closed


### Preprocessing Here

Wind direction preprocessing

Time preprocessing

In [24]:
df

,site_id,building_id,primary_use,square_feet,meter,meter_reading,timestamp,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed
0,1,147,Office,82548,0,72.0612,2016-01-01,3.8,2.4,1020.9,240.0,3.1
1,1,148,Office,174601,0,47.2000,2016-01-01,3.8,2.4,1020.9,240.0,3.1
2,1,149,Office,81881,0,53.1020,2016-01-01,3.8,2.4,1020.9,240.0,3.1
3,1,150,Office,65657,0,161.7000,2016-01-01,3.8,2.4,1020.9,240.0,3.1
4,1,151,Office,30495,0,18.8000,2016-01-01,3.8,2.4,1020.9,240.0,3.1
...,...,...,...,...,...,...,...,...,...,...,...,...
17684649,2,220,Education,122592,3,41.0299,2016-01-01,15.6,-5.6,1015.3,270.0,3.6
17684650,2,221,Education,204349,0,389.9800,2016-01-01,15.6,-5.6,1015.3,270.0,3.6
17684651,2,221,Education,204349,1,296.2590,2016-01-01,15.6,-5.6,1015.3,270.0,3.6
17684652,2,221,Education,204349,3,190.4960,2016-01-01,15.6,-5.6,1015.3,270.0,3.6
